In [ ]:
import numpy as np, os, time
import tensorflow as tf
from tensorflow import keras
from argparse import Namespace
from typing import List
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import tmodel
args: Namespace = tmodel.load_args()
tf.config.set_visible_devices([], 'GPU')             # disable GPU for this notebook

signal_index=args.signal
feature_type=args.feature_type

In [ ]:
data=tmodel.get_demo_data()
signals = data['signals']
T: np.ndarray = data['times'][signal_index]
X: np.ndarray = tmodel.get_features( T, feature_type, args )
Y: np.ndarray = signals[signal_index]

validation_split = int(0.8*X.shape[0])
Xtrain=X[:validation_split]
Xval=X[validation_split:]
Ytrain=Y[:validation_split]
Yval=Y[validation_split:]

strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")
with strategy.scope():
    model = tmodel.create_streams_model( X.shape[1], dropout_frac=args.dropout_frac, n_streams=args.nstreams )
    model.compile( optimizer=tf.keras.optimizers.Adam( learning_rate=args.learning_rate ), loss=args.loss )

In [ ]:
ckp_file = tmodel.get_ckp_file( args )
model.load_weights(ckp_file)
p0=model.predict( Xtrain, batch_size=args.batch_size )
p1=model.predict( Xval,   batch_size=args.batch_size )

In [ ]:
rfig = plt.figure(figsize=(15,5))
rax = rfig.add_axes([0.0, 0.0, 1.0, 1.0])
rax.plot(T,Y,label='truth')
rax.plot(T[:validation_split],p0[:,0],label='train prediction')
rax.plot(T[validation_split:],p1[:,0],label='val prediction')
rax.title(f'Signal {signal_index} (ftype={feature_type}): nfeatures={args.nfeatures}')
rfig.legend()
rfig.tight_layout()
rfig.show()

In [ ]:
ffig = plt.figure( figsize=(22,8) )
pax = ffig.add_axes([0.0, 0.1, 1.0, 0.9])
sax = ffig.add_axes([0.0, 0.05, 0.8, 0.04])
pax.set_title( f'Features (type {feature_type}): Nf={args.nfeatures}' )

plots: List[plt.Line2D] = []
for iF in range( X.shape[1] ):
    plots.append( pax.plot( T[:], X[:,iF], label=f'Feature {iF}', alpha=tmodel.alpha(iF,0))[0])

widget = Slider( sax, "Feature", 0, len(plots), valinit=0, valstep=1 )
widget.on_changed( lambda sval: tmodel.select_feature(plots,ffig,sval))

ffig.legend()
ffig.tight_layout()
ffig.show()